TODO: 
Clean Strings
...


In [1]:
import sys
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import nltk
from nltk.corpus import stopwords
from collections import Counter
from nltk import word_tokenize
import time, random, re, pprint
pd.set_option('display.max_columns', None)
print(sys.version_info)

sys.version_info(major=3, minor=5, micro=2, releaselevel='final', serial=0)


## Reading Data and Cleaning

In [2]:
start = time.time()
train_input = pd.read_csv('train_input.csv')
train_output = pd.read_csv('train_output.csv')
print(time.time() - start)

1.5503840446472168


In [3]:
print(train_input.shape)
train_input.head()

(165000, 2)


,id,conversation
0,0,<speaker_1> seaworld ceo steps down amid tanki...
1,1,<speaker_1> strickland chargers owner dean spa...
2,2,<speaker_1> iniesta plays keepy uppy with one ...
3,3,<speaker_1> chappie trailer #1 <number> hugh j...
4,4,<speaker_1> why the church of satan may get to...


In [4]:
delchars = ''.join(c for c in map(chr, range(256)) if not c.isalpha())
delchars = ''.join(ch for ch in delchars if ch !=' ')

def clean_text(paragraph):
    output = re.sub('\<.*?\>','', paragraph) #delete tags (between <..>)
    output = re.sub('\@.*?\s','', output) #delete usernames (words following @)
    output = re.sub('\n','', output) #delete anything following a slash
    output = output.translate(str.maketrans('','',delchars))
    return output

def rem_stopwords(word_list): 
    return list(set(word_list) - set(stopwords.words('English')))

count_stops = Counter(stopwords.words("English")*100)

def rem_stopwords_count(words_list):
    return Counter(words_list) - count_stops

In [5]:
start = time.time()
text_vector = train_input["conversation"].apply(lambda x: clean_text(x))
train_input["text"] = text_vector
train_input["words"] = train_input["text"].apply(lambda x: x.split()) 
train_input["words_count"] = train_input["words"].apply(lambda x: rem_stopwords_count(x))
train_input["output"] = train_output["category"]
#strip out all common words (stopwords, which we import from the nltk package)
print(time.time() - start)

24.713356018066406


In [6]:
train_input.head()

,id,conversation,text,words,words_count,output
0,0,<speaker_1> seaworld ceo steps down amid tanki...,seaworld ceo steps down amid tanking revenues...,"[seaworld, ceo, steps, down, amid, tanking, re...","{'com': 1, 'good': 1, 'regarding': 1, 'propaga...",news
1,1,<speaker_1> strickland chargers owner dean spa...,strickland chargers owner dean spanos and gol...,"[strickland, chargers, owner, dean, spanos, an...","{'twitter': 1, 'sachs': 2, 'strickland': 1, 't...",nfl
2,2,<speaker_1> iniesta plays keepy uppy with one ...,iniesta plays keepy uppy with one leg man yo...,"[iniesta, plays, keepy, uppy, with, one, leg, ...","{'com': 1, 'man': 1, 'almost': 1, 'walking': 1...",soccer
3,3,<speaker_1> chappie trailer #1 <number> hugh j...,chappie trailer hugh jackman sci fi comedy...,"[chappie, trailer, hugh, jackman, sci, fi, com...","{'horrifically': 1, 'intentionally': 1, 'sad':...",movies
4,4,<speaker_1> why the church of satan may get to...,why the church of satan may get to open your ...,"[why, the, church, of, satan, may, get, to, op...","{'deities': 1, 'america': 1, 'next': 1, 'praye...",politics


In [7]:
train_input["words_count"][0]

Counter({'amid': 1,
         'animals': 1,
         'announces': 1,
         'blackfish': 1,
         'campaign': 1,
         'ceo': 1,
         'changes': 1,
         'com': 1,
         'company': 1,
         'couple': 1,
         'coupled': 1,
         'criticism': 1,
         'documentary': 1,
         'good': 1,
         'killer': 1,
         'layoffs': 1,
         'least': 1,
         'like': 1,
         'little': 1,
         'make': 1,
         'maybe': 2,
         'media': 1,
         'nothing': 1,
         'number': 1,
         'outsideonline': 1,
         'pathetic': 1,
         'perception': 1,
         'propaganda': 1,
         'public': 2,
         'quo': 1,
         'regarding': 1,
         'revenues': 1,
         'safety': 1,
         'saying': 1,
         'seaworld': 2,
         'status': 1,
         'steps': 1,
         'sway': 1,
         'tanking': 1,
         'trainers': 1,
         'treatment': 1,
         'underscored': 1,
         'undisclosed': 1,
         'uphel

# Now, Train the Naive Bayes Model

In [8]:
train_input.groupby("output").size()
#Size of each of the groups

output
hockey       20861
movies       22409
nba          18422
news         21057
nfl          20106
politics     19694
soccer       21363
worldnews    21088
dtype: int64

In [ ]:
##Recursive algorithm to efficiently sum a list of counters
def mergeSum(Counter_list):
    if len(Counter_list) < 2:
        return Counter_list.iloc[0]
    
    else: 
        mid = len(Counter_list)//2

        lefthalf = mergeSum(Counter_list[:mid])
        righthalf = mergeSum(Counter_list[mid:])

        my_sum = lefthalf + righthalf
        return my_sum

In [ ]:
%%time
#Total word Counter
Total_counter = mergeSum(train_input["words_count"])

In [ ]:
%%time 
#Word Counter for each group
Groups_counter = train_input.groupby("output")["words_count"].apply(lambda x: mergeSum(x))

In [ ]:
#categories
train_input["output"].unique()
